In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [ ]:
df = pd.read_csv('/Users/maurits/Documents/Ironhack/week_05/gpw5+6/vanguard-ab-test/data/clean/dataset_TVC_MS.csv')

In [12]:
df.head()

,Unnamed: 0,client_id,level_1,reached_confirm,went_back,TD,SD,S1,S2,S3,...,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,clnt_age_rounded,age_category
0,0,555,reached_confirm,True,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,30.0,20-30
1,1,555,total_duration_minutes,NaN,NaN,2.633333,NaN,NaN,NaN,NaN,...,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,30.0,20-30
2,2,555,step_1_duration,NaN,NaN,NaN,NaN,0.533333,NaN,NaN,...,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,30.0,20-30
3,3,555,step_2_duration,NaN,NaN,NaN,NaN,NaN,1.65,NaN,...,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,30.0,20-30
4,4,555,went_back,NaN,False,NaN,NaN,NaN,NaN,NaN,...,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,30.0,20-30


In [14]:
# def convert_minutes_to_min_sec(minutes):
#     total_seconds = minutes * 60  # Convert minutes to total seconds
#     mins, sec = divmod(total_seconds, 60)
#     whole_seconds = int(sec)
#     fractional_seconds = sec - whole_seconds
#     return f"{int(mins)}:{whole_seconds}"
#     # return f"{int(mins)} minutes, {whole_seconds} seconds, {fractional_seconds:.2f} fractional seconds" 

# # Apply the function to the dataset
# td_concat['converted_time'] = td_concat['TD_control'].apply(convert_minutes_to_min_sec)
# td_concat

In [16]:
df.columns

Index(['Unnamed: 0', 'client_id', 'level_1', 'reached_confirm', 'went_back',
       'TD', 'SD', 'S1', 'S2', 'S3', 'BS', 'BS1', 'BS2', 'Variation',
       'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr', 'num_accts',
       'bal', 'calls_6_mnth', 'logons_6_mnth', 'clnt_age_rounded',
       'age_category'],
      dtype='object')

Find means of all process steps (control and test separately) Maurits
group by all
Group by sex
group by age group
group by ...

Find which steps cause most errors and for how long Maurits
group by all
Group by sex
group by age group
group by whatever


In [19]:
td_mean_control_all = df.groupby(['Variation'])['TD'].mean()
# td_mean_control_all = df.groupby(['Variation', 'gendr'])['TD'].mean()
# td_mean_control_all = df.groupby(['Variation', 'gendr', 'age_category'])['TD'].mean()

In [21]:
df_control = df[df['Variation'] == 'Control']
df_test = df[df['Variation'] == 'Test']

In [47]:
TD = df.groupby(['Variation'])['TD'].mean()
SD = df.groupby(['Variation'])['SD'].mean()
S1 = df.groupby(['Variation'])['S1'].mean()
BS1 = df.groupby(['Variation'])['BS1'].mean()
BS2 = df.groupby(['Variation'])['BS2'].mean()
S2 = df.groupby(['Variation'])['S2'].mean()
S3 = df.groupby(['Variation'])['S3'].mean()
not_grouped = pd.concat([TD, SD, S1, S2, S3, BS1, BS2], axis=1, join='outer', ignore_index=False)
# dict = dict(not_grouped)
not_grouped['BS1S1'] = not_grouped['S1'] + not_grouped['BS1']
not_grouped['BS2S2'] = not_grouped['S2'] + not_grouped['BS2']
not_grouped = not_grouped.drop(columns=['S1','S2','BS1', 'BS2'])
not_grouped = not_grouped.rename(columns={'BS1S1': 'S1', 'BS2S2': 'S2'})
not_grouped.to_csv('timenew2.csv', index=True)
not_grouped

,TD,SD,S3,S1,S2
Variation,,,,,
Control,6.367989,0.742928,1.776592,1.237517,2.286337
Test,6.147209,0.726882,1.425882,1.359484,2.306476


In [ ]:
td_mean_control = df_control.groupby(['TD']).mean()
td_mean_test = df_test.groupby(['TD']).mean()
td_concat = pd.concat([td_mean_control, td_mean_test], axis=1, join='outer', ignore_index=True)
td_concat = td_concat.rename(columns={0: 'TD_control', 1: 'TD_test'})
td_concat['TD_difference'] = td_concat['TD_test'] - td_concat['TD_control']
td_concat['TD_difference%'] = (td_concat['TD_test'] / td_concat['TD_control'])-1
td_concat = td_concat.dropna()
# td_concat = td_concat.groupby(['age_category']).mean()
td_concat

In [ ]:
sd_mean_control = df_control.groupby(['age_category'])['SD'].mean()
sd_mean_test = df_test.groupby(['age_category'])['SD'].mean()
sd_concat = pd.concat([sd_mean_control, sd_mean_test], axis=1, join='outer', ignore_index=True)
sd_concat = sd_concat.rename(columns={0: 'SD_control', 1: 'SD_test'})
sd_concat['SD_difference'] = sd_concat['SD_test'] - sd_concat['SD_control']
sd_concat['SD_difference%'] = (sd_concat['SD_test'] / sd_concat['SD_control'])-1
sd_concat = sd_concat.dropna()
sd_concat

In [ ]:
s1_mean_control = df_control.groupby(['age_category'])['S1'].mean()
s1_mean_test = df_test.groupby(['age_category'])['S1'].mean()
s1_concat = pd.concat([s1_mean_control, s1_mean_test], axis=1, join='outer', ignore_index=True)
s1_concat = s1_concat.rename(columns={0: 'S1_control', 1: 'S1_test'})
s1_concat['S1_difference'] = s1_concat['S1_test'] - s1_concat['S1_control']
s1_concat['S1_difference%'] = (s1_concat['S1_test'] / s1_concat['S1_control'])-1
s1_concat = s1_concat.dropna()
# s1_concat = s1_concat.groupby(['age_category']).mean()
s1_concat

In [ ]:
s2_mean_control = df_control.groupby(['age_category'])['S2'].mean()
s2_mean_test = df_test.groupby(['age_category'])['S2'].mean()
s2_concat = pd.concat([s2_mean_control, s2_mean_test], axis=1, join='outer', ignore_index=True)
s2_concat = s2_concat.rename(columns={0: 'S2_control', 1: 'S2_test'})
s2_concat['S2_difference'] = s2_concat['S2_test'] - s2_concat['S2_control']
s2_concat['S2_difference%'] = (s2_concat['S2_test'] / s2_concat['S2_control'])-1
s2_concat = s2_concat.dropna()
# s2_concat = s2_concat.groupby(['age_category']).mean()
s2_concat

In [ ]:
s3_mean_control = df_control.groupby(['age_category'])['S3'].mean()
s3_mean_test = df_test.groupby(['age_category'])['S3'].mean()
s3_concat = pd.concat([s3_mean_control, s3_mean_test], axis=1, join='outer', ignore_index=True)
s3_concat = s3_concat.rename(columns={0: 'S3_control', 1: 'S3_test'})
s3_concat['S3_difference'] = s3_concat['S3_test'] - s3_concat['S3_control']
s3_concat['S3_difference%'] = (s3_concat['S3_test'] / s3_concat['S3_control'])-1
s3_concat = s3_concat.dropna()
# s3_concat = s3_concat.groupby(['age_category']).mean()
s3_concat

In [ ]:
# No data
bs_mean_control = df_control.groupby(['age_category'])['BS'].mean()
bs_mean_test = df_test.groupby(['age_category'])['BS'].mean()
bs_concat = pd.concat([bs_mean_control, bs_mean_test], axis=1, join='outer', ignore_index=True)
bs_concat = bs_concat.rename(columns={0: 'BS_control', 1: 'BS_test'})
bs_concat['BS_difference'] = bs_concat['BS_test'] - bs_concat['BS_control']
bs_concat['BS_difference%'] = (bs_concat['BS_test'] / bs_concat['BS_control'])-1
bs_concat = bs_concat.dropna()
# bs_concat = bs_concat.groupby(['age_category']).mean()
bs_concat

In [ ]:
bs1_mean_control = df_control.groupby(['age_category'])['BS1'].mean()
bs1_mean_test = df_test.groupby(['age_category'])['BS1'].mean()
bs1_concat = pd.concat([bs1_mean_control, bs1_mean_test], axis=1, join='outer', ignore_index=True)
bs1_concat = bs1_concat.rename(columns={0: 'BS1_control', 1: 'BS1_test'})
bs1_concat['BS1_difference'] = bs1_concat['BS1_test'] - bs1_concat['BS1_control']
bs1_concat['BS1_difference%'] = (bs1_concat['BS1_test'] / bs1_concat['BS1_control'])-1
bs1_concat = bs1_concat.dropna()
# bs1_concat = bs1_concat.groupby(['age_category']).mean()
bs1_concat

In [ ]:
bs2_mean_control = df_control.groupby(['age_category'])['BS2'].mean()
bs2_mean_test = df_test.groupby(['age_category'])['BS2'].mean()
bs2_concat = pd.concat([bs2_mean_control, bs2_mean_test], axis=1, join='outer', ignore_index=True)
bs2_concat = bs2_concat.rename(columns={0: 'BS2_control', 1: 'BS2_test'})
bs2_concat['bs2_difference'] = bs2_concat['BS2_test'] - bs2_concat['BS2_control']
bs2_concat['bs2_difference%'] = (bs2_concat['BS2_test'] / bs2_concat['BS2_control'])-1
bs2_concat = bs2_concat.dropna()
# bs2_concat = bs2_concat.groupby(['age_category']).mean()
bs2_concat

In [ ]:
# Concatinate all means
process_step_means = pd.concat([td_concat, sd_concat, s1_concat, s2_concat, s3_concat, bs1_concat, bs2_concat], axis=1, join='outer', ignore_index=False)
process_step_means

In [ ]:
process_step_means.to_csv('process_step_means_groupedbyage.csv', index=True)

In [ ]:
sd_concat = pd.concat([td_concat, sd_concat, s1_concat, s2_concat, s3_concat, bs1_concat, bs2_concat], axis=1, join='outer', ignore_index=False)
sd_concat